In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
def parse_z(x, eps=1e-6):
    if pd.isna(x):
        return np.nan
    s = str(x).strip().upper()

    # Casos especiales que se mantienen tal cual
    codigos_especiales = {"DA", "DC", "DG", "N", "NB", "NN", "O", "R", "S", "SA"}
    if s in codigos_especiales:
        return s  # mantener como está
        
    sin_dato_set = {"S/A", "SIN DATO", "NA", "N/A", ""}
    if s in sin_dato_set:
        return np.nan

    # Normalizar separadores y signos "raros"
    s = s.replace(",", ".")          # coma -> punto decimal
    s = s.replace(" ", "")           # espacios
    s = s.replace("+-", "")          # +- -> 
    s = s.replace("++", "+")         # ++ -> +
    s = s.replace("--", "-")         # - -> -
    s = s.replace("+>", ">+")
    s = s.replace("+<", "<+")
    s = s.replace("1+", "+1")        # 1+ -> +1 

    op = None
    if s.startswith(">"):
        op = ">"
        s = s[1:]
    elif s.startswith("<"):
        op = "<"
        s = s[1:]

   # Quitar un "+", +1" -> "1"
    if s.startswith("+"):
        s = s[1:]

    # Extraer número (permite signo y decimales)
    m = re.search(r"^[+-]?\d+(\.\d+)?$", s)
    if not m:
        # Valida si se limpio
        s2 = re.sub(r"[^\d\.\-+]", "", s)
        m2 = re.search(r"^[+-]?\d+(\.\d+)?$", s2)
        if not m2:
            return np.nan
        s = s2

    try:
        val = float(s)
    except:
        return np.nan

    # Respetar > y < desplazando un epsilon
    if op == ">":
        val = val + eps
    elif op == "<":
        val = val - eps

    return val

#categorizacion
def categoria_PE(z):
    if isinstance(z, str):   # conservar códigos especiales
        return z
    """P/E: DESN (< -2) | N [-2, +2] | SOBREPESO (> +2)"""
    if pd.isna(z):
        return "NULL"
    if z < -2:
        return "R"
    if z <= 2:
        return "N"
    return "S"  # z > +2

def categoria_PT(z):
    if isinstance(z, str):   # conservar códigos especiales
        return z
    if pd.isna(z):
        return "NULL"
    if z < -3:
        return "DA"
    if z < -2:
        return "R"
    if z < -1:
        return "N"
    if z < 1:
        return "N"
    if z < 2:
        return "N"
    if z < 3:
        return "S"
    return "O"  # z >= +3

In [ ]:
# Crear columnas numéricas limpias (z-score) sin alterar las originales
df["_TE_z"] = df["T/E"].apply(parse_z)
df["_PE_z"] = df["P/E"].apply(parse_z)
df["_PT_z"] = df["P/T"].apply(parse_z)

# Crear las 3 columnas categóricas nuevas
df["T/E_cat"] = df["_TE_z"].apply(categoria_TE)
df["P/E_cat"] = df["_PE_z"].apply(categoria_PE)
df["P/T_cat"] = df["_PT_z"].apply(categoria_PT)

In [ ]:
print(df[["T/E","P/E","P/T","T/E_cat","P/E_cat","P/T_cat"]])
